<a href="https://colab.research.google.com/github/Ahny678/Performance-Analysis-of-Lightweight-CNN-models-/blob/main/PERFORMANCE_ANALYSIS_USING_BASIC_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from google.colab import drive
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pickle


In [ ]:

# Mount Google Drive
drive.mount('/content/drive')
# Relax TensorFlow error handling
tf.data.experimental.enable_debug_mode()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define dataset paths
base_dir = '/content/drive/MyDrive/PERFORMANCECROPDATASET'
maize_dir = os.path.join(base_dir, 'Maize')

rice_dir = os.path.join(base_dir, 'Rice')
sorghum_dir = os.path.join(base_dir, 'Sorghum')

# Image parameters
IMG_SIZE = (28, 28)  # Updated to match architecture
BATCH_SIZE = 32

# Output directory for saving results
output_base_dir = '/content/drive/MyDrive/PERFORMANCECROPDATASET_Hyper_Results'
os.makedirs(output_base_dir, exist_ok=True)


In [ ]:
# Data augmentation for training
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomTranslation(0.1, 0.1),
])

In [ ]:
# Function to load dataset
def load_dataset(dataset_dir, split, shuffle=False):
    # image_dataset_from_directory automatically assigns labels based on directory names
    # and returns class_names as an attribute of the dataset.
    dataset = tf.keras.utils.image_dataset_from_directory(
        os.path.join(dataset_dir, split),
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=shuffle,
        label_mode='categorical',
        color_mode='grayscale'  # 1-channel input this time since its simple cnn
    )

    # Access class_names from the dataset object
    class_names = dataset.class_names

    # Normalize to [0, 1]
    dataset = dataset.map(lambda x, y: (x / 255.0, y), num_parallel_calls=tf.data.AUTOTUNE)
    if split == 'train':
        # Ensure data augmentation is applied only during training
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
                             num_parallel_calls=tf.data.AUTOTUNE)
    # Convert to float32 for model compatibility
    dataset = dataset.map(lambda x, y: (tf.cast(x, tf.float32), y),
                         num_parallel_calls=tf.data.AUTOTUNE)
    # Cache and prefetch for performance
    dataset = dataset.cache().prefetch(tf.data.AUTOTUNE)
    return dataset, class_names # Return both dataset and class_names


BELOW COMPUTE CLASS WEIGHTS FUNCTION REWRITTEN BY COLAB GEMINI DUE TO TF ERRORS...

In [ ]:

# Function to compute class weights using dataset's class names
def compute_class_weights(dataset_dir, split, dataset_class_names):
    class_counts = {}
    split_dir = os.path.join(dataset_dir, split)

    # Collect labels based on the actual class names and their integer index mapping
    # as determined by image_dataset_from_directory
    labels = []
    for i, cls in enumerate(dataset_class_names):
        cls_dir = os.path.join(split_dir, cls)
        if os.path.exists(cls_dir):
            # List all files in the directory, filtering out non-image files if necessary
            # For simplicity, we count all entries assuming only image files or
            # directories are present under class directories.
            # A more robust approach would involve checking file extensions.
            count = len([name for name in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, name))])
            class_counts[cls] = count
            labels.extend([i] * count) # Use the index 'i' corresponding to the class name
        else:
             print(f"Warning: Directory for class '{cls}' not found at {cls_dir}. Skipping.")

    if not labels:
        print(f"Warning: No samples found in {os.path.join(dataset_dir, split)}. Cannot compute class weights.")
        return {}

    # Compute class weights using the actual integer labels found
    unique_labels = np.unique(labels)

    # compute_class_weight handles cases where some classes have zero samples in 'y'.
    # It computes weights only for classes present in 'y'.
    # The 'classes' argument should contain all possible class labels (0 to num_classes-1).
    full_range_classes = list(range(len(dataset_class_names)))

    # Check if unique_labels cover all expected classes
    if len(unique_labels) < len(full_range_classes):
        print(f"Warning: Training data does not contain samples for all classes defined by dataset structure. Missing labels: {set(full_range_classes) - set(unique_labels)}")
        # In this case, compute_class_weight will only return weights for classes present in 'y'.
        # We need to create a dictionary mapping from the full range of class indices
        # to the computed weights, filling in missing weights (e.g., with 0 or a small value,
        # though 0 is safer as it prevents the model from training on non-existent classes).
        # Let's re-think how compute_class_weight should be used here.
        # The function expects 'classes' to be the set of all possible class labels in 'y'.
        # If 'y' contains labels [0, 1, 3] and classes=[0, 1, 2, 3], it will compute weights
        # for 0, 1, and 3. The output is an array corresponding to the order of 'classes'.
        # We need to ensure 'classes' argument is correct. It should be the unique labels found in 'y'.
        # Or, if we want weights for all possible classes, we need to provide 'y' with
        # representatives of all classes, which is not feasible from just the file counts.

        # Let's use the unique labels found in the directory scan.
        weights_array = compute_class_weight(
            class_weight='balanced',
            classes=unique_labels, # Use the unique labels found in the data
            y=labels
        )
        # Create a dictionary mapping the found unique labels to their weights
        weight_dict = dict(zip(unique_labels, weights_array))

        # Important: Keras model.fit 'class_weight' expects a dictionary mapping
        # from CLASS INDEX (integer) to weight. The class indices should correspond
        # to the order of class names returned by image_dataset_from_directory.
        # So, the weight_dict should map 0 -> weight for class_names[0], 1 -> weight
        # for class_names[1], etc., regardless of whether that class was present in 'labels'.
        # This means we need to compute weights for all potential classes if we want
        # a weight_dict that maps indices 0..num_classes-1 to weights.

        # Let's revise: Compute weights for all classes based on their counts.
        # The 'labels' list already represents the sample distribution across classes
        # based on the directory structure. The indices in 'labels' correspond to
        # the order in `dataset_class_names`.
        all_class_indices = list(range(len(dataset_class_names)))
        # compute_class_weight expects 'classes' to contain all unique labels in 'y'.
        # If we want weights for all possible output classes (defined by dataset_class_names),
        # we should pass those indices. If a class index in `all_class_indices` is not present
        # in `labels`, compute_class_weight might raise an error or handle it based on
        # its implementation. A safer approach is to only include classes present in `labels`
        # in the `classes` argument, and then construct the weight dictionary for all
        # possible classes, potentially assigning a default weight (e.g., 0 or 1) to missing ones.

        # Let's stick to computing weights *only* for the classes actually found in the training data
        # based on the directory scan, and then map these weights to the correct class indices.
        # The `labels` list contains the integer indices corresponding to `dataset_class_names`.
        # So, the unique labels in `labels` are exactly the indices of the classes present.
        weights_array = compute_class_weight(
            class_weight='balanced',
            classes=unique_labels, # These are the indices of classes present
            y=labels
        )
        # Create the weight dictionary mapping class index (as found in 'labels') to weight
        weight_dict = dict(zip(unique_labels, weights_array))

        # Now, create a complete weight dictionary for all classes 0 to num_classes-1.
        # If a class index is not in `unique_labels`, its weight is not computed.
        # We should add these missing classes to the dictionary, perhaps with a weight of 0.
        # However, using a weight of 0 for missing classes might be problematic if those
        # classes actually exist in the validation/test sets.
        # A more robust approach is to ensure the training set has at least one sample
        # for each class that is expected to be in the dataset.
        # Given the current setup, compute_class_weight returns weights for `unique_labels`.
        # The `class_weight` argument in model.fit expects weights for classes 0 to num_classes-1.
        # So, we need to map the computed weights for `unique_labels` to the full range of indices.
        full_weight_dict = {i: 0.0 for i in range(len(dataset_class_names))} # Initialize all weights to 0
        for idx, weight in weight_dict.items():
            full_weight_dict[idx] = weight # Assign computed weights for existing classes

        weight_dict_to_return = full_weight_dict

    else:
        # All classes expected based on `dataset_class_names` were found in `labels`.
         weights_array = compute_class_weight(
            class_weight='balanced',
            classes=unique_labels, # Use the unique labels found
            y=labels
        )
         # Create the weight dictionary mapping class index to weight
         weight_dict_to_return = dict(zip(unique_labels, weights_array))

    return weight_dict_to_return

In [ ]:
# Load datasets and capture class names for each crop
maize_train, maize_class_names = load_dataset(maize_dir, 'train', shuffle=True)
maize_val, _ = load_dataset(maize_dir, 'validation') # We only need class names from train split
maize_test, _ = load_dataset(maize_dir, 'test')

rice_train, rice_class_names = load_dataset(rice_dir, 'train', shuffle=True)
rice_val, _ = load_dataset(rice_dir, 'validation')
rice_test, _ = load_dataset(rice_dir, 'test')

sorghum_train, sorghum_class_names = load_dataset(sorghum_dir, 'train', shuffle=True)
sorghum_val, _ = load_dataset(sorghum_dir, 'validation')
sorghum_test, _ = load_dataset(sorghum_dir, 'test')

# Compute class weights using the captured class names
maize_class_weights = compute_class_weights(maize_dir, 'train', maize_class_names)
rice_class_weights = compute_class_weights(rice_dir, 'train', rice_class_names)
sorghum_class_weights = compute_class_weights(sorghum_dir, 'train', sorghum_class_names)


Found 2524 files belonging to 4 classes.
Found 836 files belonging to 4 classes.
Found 422 files belonging to 4 classes.
Found 1830 files belonging to 6 classes.
Found 389 files belonging to 6 classes.
Found 400 files belonging to 6 classes.
Found 5439 files belonging to 7 classes.
Found 1560 files belonging to 7 classes.
Found 782 files belonging to 7 classes.


In [ ]:
# Define the CNN model based on  DATASET2 architecture from the article
def create_model(num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(28, 28, 1)),

        # First Conv Block
        tf.keras.layers.Conv2D(32, (3, 3), strides=1, padding='same'),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPooling2D((2, 2), strides=1, padding='same'),

        # Second Conv Block
        tf.keras.layers.Conv2D(32, (3, 3), strides=1, padding='same'),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPooling2D((2, 2), strides=1, padding='same'),

        # Third Conv Block
        tf.keras.layers.Conv2D(64, (3, 3), strides=1, padding='same'),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPooling2D((2, 2), strides=1, padding='same'),

        # Flatten
        tf.keras.layers.Flatten(),

        # Fully Connected Layer
        tf.keras.layers.Dense(64),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Dropout(0.5),

        # Output Layer
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
# Function to save training history plot
def save_training_history(history, crop_name, output_dir):
    plt.figure(figsize=(12, 4))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'{crop_name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{crop_name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{crop_name}_training_history.png'))
    plt.close()


In [ ]:

# Function to save confusion matrix
def save_confusion_matrix(y_true, y_pred, class_names, crop_name, output_dir):
    cm = confusion_matrix(np.argmax(y_true, axis=1), np.argmax(y_pred, axis=1))
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'{crop_name} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{crop_name}_confusion_matrix.png'))
    plt.close()

In [ ]:

# Update the crops list to use the captured class names
crops = [
    {
        'name': 'Maize',
        'train_ds': maize_train,
        'val_ds': maize_val,
        'test_ds': maize_test,
        'class_weights': maize_class_weights,
        'num_classes': len(maize_class_names), # Ensure num_classes matches the captured list
        'class_names': maize_class_names
    },
    {
        'name': 'Rice',
        'train_ds': rice_train,
        'val_ds': rice_val,
        'test_ds': rice_test,
        'class_weights': rice_class_weights,
        'num_classes': len(rice_class_names),
        'class_names': rice_class_names
    },
    {
        'name': 'Sorghum',
        'train_ds': sorghum_train,
        'val_ds': sorghum_val,
        'test_ds': sorghum_test,
        'class_weights': sorghum_class_weights,
        'num_classes': len(sorghum_class_names),
        'class_names': sorghum_class_names
    }
]

In [ ]:
import gc
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os

def train_and_save_model(crop):
    """Train and save model without test evaluation"""
    crop_output_dir = os.path.join(output_base_dir, crop['name'])
    os.makedirs(crop_output_dir, exist_ok=True)

    print(f"\nTraining model for {crop['name']}...")

    # Create and train the model
    model = create_model(crop['num_classes'])

    history = model.fit(
        crop['train_ds'],
        validation_data=crop['val_ds'],
        epochs=30,
        class_weight=crop['class_weights'],
        verbose=1
    )

    # Save training history
    save_training_history(history, crop['name'], crop_output_dir)

    # Save training history data as JSON
    with open(os.path.join(crop_output_dir, f'{crop["name"]}_history.json'), 'w') as f:
        json.dump(history.history, f)

    # Save the model FIRST before any test evaluation
    model.save(os.path.join(crop_output_dir, f'{crop["name"]}_model.h5'))
    print(f"Model for {crop['name']} saved in {crop_output_dir}")

    return model, history

In [ ]:
def evaluate_on_test_set(model, crop):
    """Safe evaluation on test set"""
    crop_output_dir = os.path.join(output_base_dir, crop['name'])

    try:
        print(f"\nAttempting test evaluation for {crop['name']}...")

        # Safe test iteration
        test_images, test_labels = [], []
        for batch in crop['test_ds']:
            try:
                images, labels = batch
                test_images.append(images.numpy())
                test_labels.append(labels.numpy())
            except Exception as e:
                print(f"Skipped a batch due to error: {str(e)}")
                continue

        if test_images:
            test_images = np.concatenate(test_images, axis=0)
            test_labels = np.concatenate(test_labels, axis=0)

            test_predictions = model.predict(test_images)

            # Save classification report
            report = classification_report(
                np.argmax(test_labels, axis=1),
                np.argmax(test_predictions, axis=1),
                target_names=crop['class_names'],
                output_dict=True
            )
            with open(os.path.join(crop_output_dir, f'{crop["name"]}_classification_report.json'), 'w') as f:
                json.dump(report, f)

            # Save confusion matrix
            save_confusion_matrix(test_labels, test_predictions, crop['class_names'], crop['name'], crop_output_dir)

            print(f"Test evaluation completed for {crop['name']}")
        else:
            print(f"Warning: No valid test batches processed for {crop['name']}")

    except Exception as e:
        print(f"Test evaluation failed for {crop['name']}: {str(e)}")



In [ ]:
# Process Maize
maize_model, maize_history = train_and_save_model(crops[0])
evaluate_on_test_set(maize_model, crops[0])
del maize_model, maize_history
gc.collect()
tf.keras.backend.clear_session()



In [ ]:
 # Process Rice
rice_model, rice_history = train_and_save_model(crops[1])
evaluate_on_test_set(rice_model, crops[1])
del rice_model, rice_history
gc.collect()
tf.keras.backend.clear_session()




Training model for Rice...
Epoch 1/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 633s 11s/step - accuracy: 0.1608 - loss: 1.9607 - val_accuracy: 0.1671 - val_loss: 1.7905
Epoch 2/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 14s 244ms/step - accuracy: 0.2012 - loss: 1.7872 - val_accuracy: 0.2596 - val_loss: 1.7274
Epoch 3/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 14s 242ms/step - accuracy: 0.2273 - loss: 1.7274 - val_accuracy: 0.3265 - val_loss: 1.6482
Epoch 4/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 13s 226ms/step - accuracy: 0.1917 - loss: 1.7931 - val_accuracy: 0.1671 - val_loss: 1.7910
Epoch 5/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 13s 220ms/step - accuracy: 0.1744 - loss: 1.7900 - val_accuracy: 0.1877 - val_loss: 1.7811
Epoch 6/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 14s 237ms/step - accuracy: 0.1840 - loss: 1.7869 - val_accuracy: 0.3265 - val_loss: 1.7135
Epoch 7/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 14s 244ms/step - accuracy: 0.2523 - loss: 1.7053 - val_accuracy: 0.3008 - val_loss: 1.6176
Epoch 8/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 14s 240ms/step - accuracy: 0.299

Model for Rice saved in /content/drive/MyDrive/PERFORMANCECROPDATASET_Results/Rice


In [ ]:
# Process Sorghum
sorghum_model, sorghum_history = train_and_save_model(crops[2])
evaluate_on_test_set(sorghum_model, crops[2])
del sorghum_model, sorghum_history
gc.collect()
tf.keras.backend.clear_session()